In [4]:
folder = "C:\Users\dipic\Google Drive\Gaming\Pen and Paper RPGs\Genesys\Adventure Design\\"
CreatureCatalogue = open(folder + "Creature Catalogue.txt","r")

for line_num,line_full in enumerate(CreatureCatalogue):
    #Replacing tabs with spaces, then squashing to single spaces makes transforming a little easier
    line = line_full.replace('\t', ' ')
    while line.find("  ") > -1:
        line = line.replace('  ', ' ')
    #I also want to remove trailing spaces from lines
    line = line.replace(' \n','\n')
        
    if line != "\n":
        print line_num
        print line
    if line_num >= 15:
        break

0
Camel (Rival) – Foul tempered and belligerent, camels nevertheless make excellent mounts and beasts of burden in arid desert environments.

1
Brawn: 3 Agility: 1 Intelligence: 1

2
Cunning: 1 Willpower: 2 Presence: 1

3
Soak: 3 Wounds: 14 M/R Defense: 0/0

4
Skills: Athletics 2, Brawl 1, Survival 1

5
Talents: None

6
Abilities: Silhouette 2

8
Trained Mount 1 (add 1 Boost to a rider's Riding checks while mounted)

10
Sand Walker (remove 1 Setback from any checks made to traverse sandy or desert terrain)

12
Beast of Burden 5 (add 5 to encumbrance threshold)

14
Equipment: Hooves (Brawl; Damage: 5; Critical: 5; Range [Engaged])



In [9]:
#This works for rivals. I'll copy it for a nemesis just to make sure I don't break it
folder = "C:\Users\dipic\Google Drive\Gaming\Pen and Paper RPGs\Genesys\Adventure Design\\"
JSONformat = open(r"Creature Catalogue JSON.json","w+")
JSONtext = '\t{\n\t\t"name": "'

char_list = ["Brawn","Agility","Intelligence","Cunning","Willpower","Presence"]
    
CreatureCatalogue = open(folder + "Creature Catalogue.txt","r")

#Start at 999999 so when I use the initial between it's always false, but then after we set the ability
#line it works
ability_line_num = 999999

for line_num,line_full in enumerate(CreatureCatalogue):
    #print line_num
    line = line_full
    #Firstly, I want to replace all tabs and double-spaces with single spaces to make parsing easier
    line = line_full.replace('\t', ' ')
    while line.find("  ") > -1:
        line = line.replace('  ', ' ')
    #I also want to remove trailing spaces from lines
    line = line.replace(' \n','\n')
    
    #For the first line, we know it starts with the name, type, and description
    #print('Line Number: ' + str(line_num))
    #Print where I expect the characteristic text to be
    #print(line[:line.find(":")] in char_list)
    if line_num == 0:
        #Grab the character name
        p1 = line.find(" (")
        JSONtext += line[:p1] + '",\n\t\t"type": "'

        #Grab the type
        p2 = line.find(")")
        JSONtext += line[p1+2:p2] + '",\n\t\t"description": "'

        #Grab the description
        p1 = p2 + 3
        p2 = line.find("\n")
        JSONtext += line[p1+3:p2] + '",\n\t\t"characteristics": {' #\n\t\t\t"Brawn": '
        
    if line[:line.find(":")] in char_list:
        #Loop through characteristics
        for c in char_list:
            #print(line.find(c))
            #If we can find that characteristic on this line
            if line.find(c) > -1:
                p1 = line.find(c) + len(c) + 2
                #print(p1)
                if c == "Presence":
                    JSONtext += '\n\t\t\t"' + c + '": ' + line[p1:p1+1] + '\n\t\t},\n\t\t"derived": {'
                else:
                    #print('\n\t\t\t"' + c + '": ' + line[p1:p1+1] + ',')
                    #p2 = re.search('[0-9]', line)
                    JSONtext += '\n\t\t\t"' + c + '": ' + line[p1:p1+1] + ','
            
    if line[:line.find(":")] in ["Soak","Wounds", "M/R Defense"]:
        #Loop through derived characteristics (for rivals at least)
        for d in ["Soak","Wounds", "M/R Defense"]:
            if line.find(d) > -1:
                p1 = line.find(d) + len(d) + 2
                p2 = line.find(' ',p1)
                #I need to use [,] instead of /
                if d == "M/R Defense":
                    #I'm assuming the defence will always be single digits
                    #print('\n\t\t\t"defence": [' + line[p1:p1+1] + ',' + line[p1+2:p1+3] + ']')
                    JSONtext += '\n\t\t\t"defence": [' + line[p1:p1+1] + ',' + line[p1+2:p1+3] + ']\n\t\t},\n\t\t"skills": {'
                else:
                    #print('\n\t\t\t"' + d.lower() + '": ' + line[p1:p2] + ',')
                    JSONtext += '\n\t\t\t"' + d.lower() + '": ' + line[p1:p2] + ','
                    
    if line[:line.find(":")] == "Skills":
        #Skills: Athletics 1, Brawl 2, Resilience 2
        SkillSplit = line.replace('\n','')[line.find(":")+2:].split(", ")
        for s in SkillSplit:
            #I'm assuming the skills will always be single digits
            JSONtext += '\n\t\t\t"' + s[:s.find(" ")] + '": ' + s[s.find(" ")+1:s.find(" ")+2] + ","
        #Now take off that final comma
        #I should check here for talents, probably, but let's go one example at a time
        JSONtext = JSONtext[:len(JSONtext)-1] + '\n\t\t},\n\t\t"abilities": ['
        
    if line[:line.find(":")] == "Abilities":
        ability_line_num = line_num
        
    #This needs to handle the next few lines of abilities
    if line_num >= ability_line_num and line[:line.find(":")] != "Equipment" and line != "\n":
        print line
        #I think this split is pointless as they are always on separate lines
        #AbilitySplit = line.replace('\n','').replace('Abilities: ', '').split(", ")
        Ability = line.replace('\n','').replace('Abilities: ', '')
        #print AbilitySplit
        #for a in AbilitySplit:
        #    JSONtext += '"' + a + '", '
        JSONtext += '"' + Ability + '", '
        
    if line[:line.find(":")] == "Equipment":
        ability_line_num = 999999
        #Now take off that final comma and space from the abilities
        JSONtext = JSONtext[:len(JSONtext)-2] + '],\n\t\t"weapons": ['
        #WeaponSplit = line.replace('\n','')[line.find(":")+2:].split("), ")
        #Trying to keep the delimiters
        d = "), "
        #To make this delimiting easier
        newlinetext = line.replace('\n','')[line.find(":")+2:] + ", "
        #print newlinetext
        WeaponSplit =  [e+")" for e in newlinetext.split("), ")]
        for w in WeaponSplit:
            #First, make sure it's a weapon
            #print w
            if w.find("Damage") > -1:
                JSONtext += '\n\t\t\t{\n\t\t\t\t"name": "' + w[:w.find(" (")] + '",\n\t\t\t\t'
                WeaponStatSplit = w[w.find("(")+1:].split("; ")
                #for ws in WeaponStatSplit:
                    #print ws
                #First is skill
                JSONtext += '"skill": "' + WeaponStatSplit[0] + '",\n\t\t\t\t'
                #Then the damage
                JSONtext += '"damage": ' + WeaponStatSplit[1][WeaponStatSplit[1].find(" ")+1:] + ',\n\t\t\t\t'
                #Then the critical rating
                JSONtext += '"critical": ' + WeaponStatSplit[2][WeaponStatSplit[2].find(" ")+1:] + ',\n\t\t\t\t'
                #Then the range
                JSONtext += '"range": "' + WeaponStatSplit[3][WeaponStatSplit[3].find("[")+1:WeaponStatSplit[3].find("]")] + '",\n\t\t\t\t'
                #Now get the special qualities
                Qualities = WeaponStatSplit[3][WeaponStatSplit[3].find("]")+3:].split(", ")
                JSONtext += '"qualities": ['
                for q in Qualities:
                    #print q
                    JSONtext += '"' + q + '",'
                #Now take off that final comma after the last quality
                JSONtext = JSONtext[:len(JSONtext)-1] + ']\n\t\t\t},'
                
        #Once all of the weapons are done, add the gear line and then loop through again
        #But first, take off the final comma from the last weapon
        #Somehow, only run this if there is an item of gear without damage...
        '''
        JSONtext = JSONtext[:len(JSONtext)-1] + '\n\t\t],\n\t\t"gear": ['
        for w in WeaponSplit:
            if w.find("Damage") == -1 and w.find("(") > -1:
                JSONtext += '"' + w + '",'
        '''
        #Now take off that final comma after the last item
        #Plus, add the tags. I won't try species or category tags yet, just source and CR
        #Worry about gear later
        #JSONtext = JSONtext[:len(JSONtext)-1] + '],\n\t\t"tags": [' + \
        JSONtext = JSONtext[:len(JSONtext)-1] + '\n\t\t],\n\t\t"tags": [' + \
                        '"source:Creature Catalogue", "Homebrew", "Fantasy",' + \
                        '"Combat CR:TBD", "Social CR:TBD", "General CR:TBD"]\n\t}'
    
    if line_num >= 15:
        break

print JSONtext
CreatureCatalogue.close()

Abilities: Silhouette 2

Trained Mount 1 (add 1 Boost to a rider's Riding checks while mounted)

Sand Walker (remove 1 Setback from any checks made to traverse sandy or desert terrain)

Beast of Burden 5 (add 5 to encumbrance threshold)

	{
		"name": "Camel",
		"type": "Rival",
		"description": "Foul tempered and belligerent, camels nevertheless make excellent mounts and beasts of burden in arid desert environments.",
		"characteristics": {
			"Brawn": 3,
			"Agility": 1,
			"Intelligence": 1,
			"Cunning": 1,
			"Willpower": 2,
			"Presence": 1
		},
		"derived": {
			"soak": 3,
			"wounds": 14,
			"defence": [0,0]
		},
		"skills": {
			"Athletics": 2,
			"Brawl": 1,
			"Survival": 1
		},
		"abilities": ["Silhouette 2", "Trained Mount 1 (add 1 Boost to a rider's Riding checks while mounted)", "Sand Walker (remove 1 Setback from any checks made to traverse sandy or desert terrain)", "Beast of Burden 5 (add 5 to encumbrance threshold)"],
		"weapons": [
			{
				"name": "Hooves",
				"ski

In [ ]:
#This works for nemeses

CreatureCatalogue = open(r"Creature Catalogue.txt","r")
JSONformat = open(r"Creature Catalogue JSON.json","w+")
JSONtext = '\t{\n\t\t"name": "'

char_list = ["Brawn","Agility","Intelligence","Cunning","Willpower","Presence"]
    
CreatureCatalogue = open(r"Creature Catalogue.txt","r")

for line_num,line_full in enumerate(CreatureCatalogue):
    line = line_full
    #Firstly, I want to replace all tabs and double-spaces with single spaces to make parsing easier
    line = line_full.replace('\t', ' ')
    while line.find("  ") > -1:
        line = line.replace('  ', ' ')
    #I also want to remove trailing spaces from lines
    line = line.replace(' \n','\n')
    
    #For the first line, we know it starts with the name, type, and description
    #print('Line Number: ' + str(line_num))
    #Print where I expect the characteristic text to be
    #print(line[:line.find(":")] in char_list)
    if line_num == 0:
        #Grab the character name
        p1 = line.find(" (")
        JSONtext += line[:p1] + '",\n\t\t"type": "'

        #Grab the type
        p2 = line.find(")")
        JSONtext += line[p1+2:p2] + '",\n\t\t"description": "'

        #Grab the description
        p1 = p2 + 3
        p2 = line.find("\n")
        JSONtext += line[p1+3:p2] + '",\n\t\t"characteristics": {' #\n\t\t\t"Brawn": '
        
    if line[:line.find(":")] in char_list:
        #Loop through characteristics
        for c in char_list:
            #print(line.find(c))
            #If we can find that characteristic on this line
            if line.find(c) > -1:
                p1 = line.find(c) + len(c) + 2
                #print(p1)
                if c == "Presence":
                    JSONtext += '\n\t\t\t"' + c + '": ' + line[p1:p1+1] + '\n\t\t},\n\t\t"derived": {'
                else:
                    #print('\n\t\t\t"' + c + '": ' + line[p1:p1+1] + ',')
                    #p2 = re.search('[0-9]', line)
                    JSONtext += '\n\t\t\t"' + c + '": ' + line[p1:p1+1] + ','
            
    if line[:line.find(":")] in ["Soak","Wounds", "M/R Defense"]:
        #Loop through derived characteristics (for rivals at least)
        for d in ["Soak","Wounds", "M/R Defense"]:
            if line.find(d) > -1:
                p1 = line.find(d) + len(d) + 2
                p2 = line.find(' ',p1)
                #I need to use [,] instead of /
                if d == "M/R Defense":
                    #I'm assuming the defence will always be single digits
                    #print('\n\t\t\t"defence": [' + line[p1:p1+1] + ',' + line[p1+2:p1+3] + ']')
                    JSONtext += '\n\t\t\t"defence": [' + line[p1:p1+1] + ',' + line[p1+2:p1+3] + ']\n\t\t},\n\t\t"skills": {'
                else:
                    #print('\n\t\t\t"' + d.lower() + '": ' + line[p1:p2] + ',')
                    JSONtext += '\n\t\t\t"' + d.lower() + '": ' + line[p1:p2] + ','
                    
    if line[:line.find(":")] == "Skills":
        #Skills: Athletics 1, Brawl 2, Resilience 2
        SkillSplit = line.replace('\n','')[line.find(":")+2:].split(", ")
        for s in SkillSplit:
            #I'm assuming the skills will always be single digits
            JSONtext += '\n\t\t\t"' + s[:s.find(" ")] + '": ' + s[s.find(" ")+1:s.find(" ")+2] + ","
        #Now take off that final comma
        #I should check here for talents, probably, but let's go one example at a time
        JSONtext = JSONtext[:len(JSONtext)-1] + '\n\t\t},\n\t\t"abilities": ['
        
    if line[:line.find(":")] == "Abilities":
        AbilitySplit = line.replace('\n','')[line.find(":")+2:].split(", ")
        #print AbilitySplit
        for a in AbilitySplit:
            JSONtext += '"' + a + '", '
        #Now take off that final comma and space
        JSONtext = JSONtext[:len(JSONtext)-2] + '],\n\t\t"weapons": ['
        
    if line[:line.find(":")] == "Equipment":
        #WeaponSplit = line.replace('\n','')[line.find(":")+2:].split("), ")
        #Trying to keep the delimiters
        d = "), "
        #To make this delimiting easier
        newlinetext = line.replace('\n','')[line.find(":")+2:] + ", "
        #print newlinetext
        WeaponSplit =  [e+")" for e in newlinetext.split("), ")]
        for w in WeaponSplit:
            #First, make sure it's a weapon
            #print w
            if w.find("Damage") > -1:
                JSONtext += '\n\t\t\t{\n\t\t\t\t"name": "' + w[:w.find(" (")] + '",\n\t\t\t\t'
                WeaponStatSplit = w[w.find("(")+1:].split("; ")
                #for ws in WeaponStatSplit:
                    #print ws
                #First is skill
                JSONtext += '"skill": "' + WeaponStatSplit[0] + '",\n\t\t\t\t'
                #Then the damage
                JSONtext += '"damage": ' + WeaponStatSplit[1][WeaponStatSplit[1].find(" ")+1:] + ',\n\t\t\t\t'
                #Then the critical rating
                JSONtext += '"critical": ' + WeaponStatSplit[2][WeaponStatSplit[2].find(" ")+1:] + ',\n\t\t\t\t'
                #Then the range
                JSONtext += '"range": "' + WeaponStatSplit[3][WeaponStatSplit[3].find("[")+1:WeaponStatSplit[3].find("]")] + '",\n\t\t\t\t'
                #Now get the special qualities
                Qualities = WeaponStatSplit[3][WeaponStatSplit[3].find("]")+3:].split(", ")
                JSONtext += '"qualities": ['
                for q in Qualities:
                    #print q
                    JSONtext += '"' + q + '",'
                #Now take off that final comma after the last quality
                JSONtext = JSONtext[:len(JSONtext)-1] + ']\n\t\t\t},'
                
        #Once all of the weapons are done, add the gear line and then loop through again
        #But first, take off the final comma from the last weapon
        #Somehow, only run this if there is an item of gear without damage...
        '''
        JSONtext = JSONtext[:len(JSONtext)-1] + '\n\t\t],\n\t\t"gear": ['
        for w in WeaponSplit:
            if w.find("Damage") == -1 and w.find("(") > -1:
                JSONtext += '"' + w + '",'
        '''
        #Now take off that final comma after the last item
        #Plus, add the tags. I won't try species or category tags yet, just source and CR
        #Worry about gear later
        #JSONtext = JSONtext[:len(JSONtext)-1] + '],\n\t\t"tags": [' + \
        JSONtext = JSONtext[:len(JSONtext)-1] + '\n\t\t],\n\t\t"tags": [' + \
                        '"source:Creature Catalogue", "Homebrew", "Fantasy",' + \
                        '"Combat CR:TBD", "Social CR:TBD", "General CR:TBD"]\n\t}'
    
    if line_num >= 15:
        break

print JSONtext
CreatureCatalogue.close()